In [1]:
import graphlab

In [2]:
graphlab.canvas.set_target('ipynb')

In [3]:
taps = graphlab.SFrame.read_csv("data/thousand_taps_f.csv")

This non-commercial license of GraphLab Create for academic use is assigned to jenarvaezg@gmail.com and will expire on December 12, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\JOSENR~1\AppData\Local\Temp\graphlab_server_1488211673.log.0


Finished parsing file C:\Users\José Enrique Narváez\Documents\TFM\data\thousand_taps_f.csv

Parsing completed. Parsed 100 lines in 1.41755 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[long,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,long,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,str,str,str,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\José Enrique Narváez\Documents\TFM\data\thousand_taps_f.csv

Parsing completed. Parsed 88140 lines in 1.76737 secs.

In [4]:
features = [
    "n_accel",
    
    "accel_x_mean",
    "accel_x_median",
    "accel_x_var",
    "accel_x_skewness",
    "accel_x_kurtosis",
    "accel_x_diff",
    
    "accel_y_mean",
    "accel_y_median",
    "accel_y_skewness",
    "accel_y_kurtosis",
    "accel_y_var",
    "accel_y_diff",
    
    "accel_z_mean",
    "accel_z_var",   
    "accel_z_median",
    "accel_z_skewness",
    "accel_z_kurtosis",
    "accel_z_diff",
    
    "n_gyro",
    
    "gyro_x_mean",
    "gyro_x_median",
    "gyro_x_var",
    "gyro_x_skewness",
    "gyro_x_kurtosis",
    "gyro_x_diff",
    
    "gyro_y_mean",
    "gyro_y_median",
    "gyro_y_var",
    "gyro_y_skewness",
    "gyro_y_kurtosis",
    "gyro_y_diff",
    
    "gyro_z_mean",
    "gyro_z_median",
    "gyro_z_var",
    "gyro_z_skewness",
    "gyro_z_kurtosis",
    "gyro_z_diff",
    
] 



In [5]:
train, test = taps.random_split(0.8, seed=1)

In [6]:
noise_model = graphlab.load_model("models/optimized_noise_classifier")
type_model = graphlab.load_model("models/optimized_type_classifier")
touch_model = graphlab.load_model("models/optimized_touch_model")
swipe_model = graphlab.load_model("models/optimized_swipe_model")

In [7]:
noise = test[test['noise'] == 1]
not_noise = test[test['noise'] == 0]

touches = not_noise[not_noise['type'] == "TOUCH"]
swipes = not_noise[not_noise['type'] == "SWIPE"]

# [Ceiling Analysis explained](https://d3c33hcgiwev3.cloudfront.net/19.4-ApplicationExamplePhotoOCR-CeilingAnalysisWhatPartOfThePipelineToWorkOnNext.6acb6550b22b11e49c064db6ead92550/full/540p/index.mp4?Expires=1488067200&Signature=Wlu5B9uRkREYKt0~6bVR~i6Tp5zN6k-uCRjFczQZKDu1dy6a1R55DLNWtknej1AbqtmZccxk8qDUiIJ7kT9U48llZxK1Y1BV2jqHR~iSkYK-pLmqZDCH-bDkEyMgDj-bdckYvqadcFpKj308QhIW8ihdsql-h9ehQx5DlDG5ERs_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A)

# Full model

In [8]:
predicted_not_noise = test[noise_model.predict(test) == 0]


predicted_touches = predicted_not_noise[type_model.predict(predicted_not_noise) == "TOUCH"]
predicted_swipes = predicted_not_noise[type_model.predict(predicted_not_noise) == "SWIPE"]

print "Touch final f1", touch_model.evaluate(predicted_touches)['f1_score']*100, "%"
print "Touch final confusion"
print touch_model.evaluate(predicted_touches)['confusion_matrix'].sort("count", ascending=False)
print "Swipe final f1", swipe_model.evaluate(predicted_swipes)['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_model.evaluate(predicted_swipes)['confusion_matrix'].sort("count", ascending=False)

Touch final f1 88.9046399966 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  200  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  179  |
|  CENTER-LEFT  |   CENTER-LEFT   |  176  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  167  |
|   TOP-RIGHT   |    TOP-RIGHT    |  165  |
|     CENTER    |      CENTER     |  160  |
|   TOP-CENTER  |    TOP-CENTER   |  131  |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  125  |
|    TOP-LEFT   |     TOP-LEFT    |  120  |
|     NOISE     |    TOP-RIGHT    |   4   |
+---------------+-----------------+-------+
[20 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Swipe final f1 66.4897122084 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+

# When providing real not_noise

In [9]:
predicted_touches = not_noise[type_model.predict(not_noise) == "TOUCH"]
predicted_swipes = not_noise[type_model.predict(not_noise) == "SWIPE"]

print "Touch final f1", touch_model.evaluate(predicted_touches)['f1_score']*100, "%"
print "Touch final confusion"
print touch_model.evaluate(predicted_touches)['confusion_matrix'].sort("count", ascending=False)
print "Swipe final f1", swipe_model.evaluate(predicted_swipes)['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_model.evaluate(predicted_swipes)['confusion_matrix'].sort("count", ascending=False)

Touch final f1 99.769580207 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  204  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  188  |
|  CENTER-LEFT  |   CENTER-LEFT   |  181  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  171  |
|   TOP-RIGHT   |    TOP-RIGHT    |  168  |
|     CENTER    |      CENTER     |  166  |
|   TOP-CENTER  |    TOP-CENTER   |  131  |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  129  |
|    TOP-LEFT   |     TOP-LEFT    |  123  |
|    TOP-LEFT   |    TOP-CENTER   |   2   |
+---------------+-----------------+-------+
[11 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Swipe final f1 79.8473282443 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+


# When providing real swipes and touches

In [11]:
print "Touch final f1", touch_model.evaluate(touches)['f1_score']*100, "%"
print "Touch final confusion"
print touch_model.evaluate(touches)['confusion_matrix'].sort("count", ascending=False)
print "Swipe final f1", swipe_model.evaluate(swipes)['f1_score']*100, "%"
print "Swipe final confusion"
print swipe_model.evaluate(swipes)['confusion_matrix'].sort("count", ascending=False)

Touch final f1 99.7702131023 %
Touch final confusion
+---------------+-----------------+-------+
|  target_label | predicted_label | count |
+---------------+-----------------+-------+
|  BOTTOM-RIGHT |   BOTTOM-RIGHT  |  204  |
|  CENTER-RIGHT |   CENTER-RIGHT  |  188  |
|  CENTER-LEFT  |   CENTER-LEFT   |  181  |
|  BOTTOM-LEFT  |   BOTTOM-LEFT   |  171  |
|   TOP-RIGHT   |    TOP-RIGHT    |  168  |
|     CENTER    |      CENTER     |  166  |
|   TOP-CENTER  |    TOP-CENTER   |  132  |
| BOTTOM-CENTER |  BOTTOM-CENTER  |  129  |
|    TOP-LEFT   |     TOP-LEFT    |  123  |
|    TOP-LEFT   |    TOP-CENTER   |   2   |
+---------------+-----------------+-------+
[11 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
Swipe final f1 100.0 %
Swipe final confusion
+--------------+-----------------+-------+
| target_label | predicted_label | count |
+--------------+-----------------+-------+
| BOTTO